#**유통 판매량 예측 및 재고 최적화**
## **단계3 : 모델링 및 비즈니스 평가**

# **0.미션**

* 단계1 다양한 모델링 수행
    * LSTM, CNN 등을 이용해서 다양한 시계열 모델링 수행
    * 성능 상위 2~3개 모델 선정
* 단계2 데이터 파이프라인 구축
    * test 데이터(raw data)로 부터 예측에 사용할 수 있는 형태로 데이터를 만드는 과정을 하나의 함수로 엮기
* 단계3 비즈니스 평가
    * 단계1에서 선정된 모델에 대해 test셋으로 평균 재고액 평가 수행

* 대상 상품(핵심 상품)

| Product_ID|Product_Code|SubCategory|Category|LeadTime|Price|
|----|----|----|----|----|----|
|3|DB001|Beverage|Drink|2|8|
|12|GA001|Milk|Food|3|6|
|42|FM001|Agricultural products|Grocery|3|5|


# **1.환경설정**

* 세부 요구사항
    * 구글콜랩
        * 구글 드라이브 바로 밑에 project 폴더를 만들고,
        * 데이터 파일을 복사해 넣습니다.
    - 라이브러리 로딩
        * 기본적으로 필요한 라이브러리를 import 하도록 코드가 작성되어 있습니다.
        * 필요하다고 판단되는 라이브러리를 추가하세요.


### **(1) 경로 설정**

* 구글 드라이브 연결

In [66]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [92]:
path = '/content/drive/MyDrive/KTAIVLE/6차 미니프로젝트/'

### **(2) 라이브러리 설치 및 불러오기**

In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import joblib

from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm

from keras.models import Sequential
from keras.backend import clear_session
from keras.layers import Dense, LSTM, Conv1D, Flatten, Bidirectional
from keras.optimizers import Adam

import warnings
warnings.filterwarnings("ignore")

In [69]:
# 학습곡선 함수
def dl_history_plot(history):
    plt.plot(history['loss'], label='train_err', marker = '.')
    plt.plot(history['val_loss'], label='val_err', marker = '.')

    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.grid()
    plt.show()

In [70]:
# 예측 결과 시각화
def plot_model_result(y_train, y_val, pred) :

    y_train = pd.Series(y_train)
    y_val = pd.Series(y_val)
    y_val.index = range(len(y_train), len(y_train) + len(y_val))

    pred = pd.Series(pred.reshape(-1,), index = y_val.index)

    # 전체 시각화
    plt.figure(figsize = (20,12))
    plt.subplot(2,1,1)
    plt.plot(y_train, label = 'train')
    plt.plot(y_val, label = 'val')
    plt.plot(pred, label = 'pred')
    plt.legend()
    plt.grid()

    plt.subplot(2,1,2)
    plt.plot(y_val, label = 'val')
    plt.plot(pred, label = 'pred')
    plt.legend()
    plt.grid()

    plt.show()

### **(3) 데이터 불러오기**

* **세부 요구사항**
    - 상품별로 저장한 데이터를 불러오시오.

In [71]:
# 3번 상품
temp3 = joblib.load(path + '1.data/temp3.pkl')
# 12번 상품
temp12 = joblib.load(path + '1.data/temp12.pkl')
# 42번 상품
temp42 = joblib.load(path + '1.data/temp42.pkl')

In [72]:
temp3.tail(2)

,Store_ID,Qty,Product_ID,WTI_Price,CustomerCount,LeadTime,Price,Store_Type,종가,시가,...,State_Massachusetts,State_Minnesota,State_New York,State_North Carolina,State_Ohio,State_Oklahoma,State_Pennsylvania,State_Tennessee,State_Virginia,State_Washington
Date,,,,,,,,,,,,,,,,,,,,,
2017-02-27,44,10006.0,3,54.04,3961,2,8,1,5861.90,5835.04,...,False,True,False,False,False,False,False,False,False,False
2017-02-28,44,14374.0,3,54.00,4589,2,8,1,5825.44,5852.79,...,False,True,False,False,False,False,False,False,False,False


In [73]:
temp12.tail(2)

,Store_ID,Qty,Product_ID,WTI_Price,CustomerCount,LeadTime,Price,Store_Type,종가,시가,...,State_Massachusetts,State_Minnesota,State_New York,State_North Carolina,State_Ohio,State_Oklahoma,State_Pennsylvania,State_Tennessee,State_Virginia,State_Washington
Date,,,,,,,,,,,,,,,,,,,,,
2017-02-27,44,10618.0,12,54.04,3961,2,6,1,5861.90,5835.04,...,False,True,False,False,False,False,False,False,False,False
2017-02-28,44,14400.0,12,54.00,4589,2,6,1,5825.44,5852.79,...,False,True,False,False,False,False,False,False,False,False


In [74]:
temp42.tail(2)

,Store_ID,Qty,Product_ID,WTI_Price,CustomerCount,LeadTime,Price,Store_Type,종가,시가,...,State_Massachusetts,State_Minnesota,State_New York,State_North Carolina,State_Ohio,State_Oklahoma,State_Pennsylvania,State_Tennessee,State_Virginia,State_Washington
Date,,,,,,,,,,,,,,,,,,,,,
2017-02-27,44,78.0,42,54.04,3961,2,5,1,5861.90,5835.04,...,False,True,False,False,False,False,False,False,False,False
2017-02-28,44,87.0,42,54.00,4589,2,5,1,5825.44,5852.79,...,False,True,False,False,False,False,False,False,False,False


# **2.모델링**

* **세부 요구사항**
    * 3차원 데이터 구조 만들기
        * timestep 수는 적절한 간격으로 한가지 경우만 지정을 합니다.
    * LSTM, CNN 기반 초기 모델 생성
        * 성능 보다는 코드 틀을 작성하는데 집중합시다.
        * 노드 혹은 필터 수와 크기는 초기값으로 적절하게 지정해 봅시다.
    * 데이터 파이프라인 함수 작성하기
        * input : raw data
        * output : x_train, x_val, y_train, y_val
        * 원본 데이터에서 일부를 떼어 놓고, 파이프라인 함수를 테스트 해 봅시다.

In [75]:
def temporalize(x, y, timesteps):
    output_X = []
    output_y = []
    for i in range(len(x) - timesteps + 1):
        output_X.append(x.iloc[i:i+timesteps])
        output_y.append(y.iloc[i+timesteps-1])
    return np.array(output_X), np.array(output_y)

##### 전처리 코드(학습용, 테스트용)

In [76]:
df = pd.read_csv(path + 'master_df.csv')
df_test = pd.read_csv(path + 'master_test.csv', encoding='cp949')
df['Date'] = pd.to_datetime(df['Date'] )
df_test['Date'] = pd.to_datetime(df_test['Date'] )
df.set_index('Date', inplace=True)
df_test.set_index('Date', inplace=True)
df.drop(columns='Unnamed: 0', inplace=True)
df_test.drop(columns='Unnamed: 0', inplace=True)
df.drop(columns='Product_Code', inplace=True)
df_test.drop(columns='Product_Code', inplace=True)

subCategory_dummies = pd.get_dummies(df['SubCategory'], prefix='SubCategory')
category_dummies = pd.get_dummies(df['Category'], prefix='Category')
city_dummies = pd.get_dummies(df['City'], prefix='City')
state_dummies = pd.get_dummies(df['State'], prefix='State')

df = pd.concat([df, subCategory_dummies, category_dummies, city_dummies, state_dummies], axis=1)
df.drop(['SubCategory', 'Category', 'City', 'State'], axis=1, inplace=True)

subCategory_dummies = pd.get_dummies(df_test['SubCategory'], prefix='SubCategory')
category_dummies = pd.get_dummies(df_test['Category'], prefix='Category')
city_dummies = pd.get_dummies(df_test['City'], prefix='City')
state_dummies = pd.get_dummies(df_test['State'], prefix='State')

df_test = pd.concat([df_test, subCategory_dummies, category_dummies, city_dummies, state_dummies], axis=1)
df_test.drop(['SubCategory', 'Category', 'City', 'State'], axis=1, inplace=True)

In [77]:
# 데이터 준비 함수
def prepare_data(df, product_id, look_back=2):
    # 해당 제품 ID와 스토어 ID 44에 대한 데이터 필터링
    filtered_df = df[(df['Store_ID'] == 44) & (df['Product_ID'] == product_id)]

    # 'Qty'를 제외한 나머지 컬럼을 특성으로 사용
    X = filtered_df.drop(['Qty'], axis=1).values
    Y = filtered_df['Qty'].values

    # 데이터 정규화
    scaler_X = MinMaxScaler(feature_range=(0, 1))
    scaler_Y = MinMaxScaler(feature_range=(0, 1))

    X_scaled = scaler_X.fit_transform(X)
    Y_scaled = scaler_Y.fit_transform(Y.reshape(-1,1)).flatten()

    # LSTM 모델 입력을 위한 데이터 준비
    X_data, Y_data = [], []
    for i in range(len(X_scaled)-look_back):
        X_data.append(X_scaled[i:(i+look_back)])
        Y_data.append(Y_scaled[i + look_back])
    X_data, Y_data = np.array(X_data), np.array(Y_data)

    return X_data, Y_data, scaler_X, scaler_Y

## **(1) LSTM 모델링**

##### 모델링 코드

In [99]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score
import numpy as np

# LSTM 모델
def train_and_predict(df, product_id, test_size=0.2, random_state=None):
    X, Y, scaler_X, scaler_Y = prepare_data(df, product_id)  # prepare_data 함수는 데이터 준비를 담당합니다.

    X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, test_size=test_size, random_state=random_state)

    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
        Dropout(0.2),
        LSTM(50, return_sequences=True),
        Dropout(0.2),
        LSTM(50, return_sequences=True),
        Dropout(0.2),
        LSTM(50),
        Dropout(0.2),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')

    model.fit(X_train, Y_train, epochs=100, batch_size=1, verbose=1, callbacks=[EarlyStopping(monitor='loss', patience=10)])

    # 모델 저장
    model_filename = 'model_{}.h5'.format(product_id)  # 제품 ID를 포함한 파일명 생성
    model.save(model_filename)  # 모델 저장

    # 예측 및 예측 값 역정규화
    prediction_scaled = model.predict(X_valid)
    prediction = scaler_Y.inverse_transform(prediction_scaled)

    Y_test_inv = scaler_Y.inverse_transform(Y_valid.reshape(-1, 1))

    rmse = np.sqrt(mean_squared_error(Y_test_inv, prediction))
    mae = mean_absolute_error(Y_test_inv, prediction)
    mape = mean_absolute_percentage_error(Y_test_inv, prediction)
    r2 = r2_score(Y_test_inv, prediction)
    mse = mean_squared_error(Y_test_inv, prediction)

    return {
        "prediction": prediction,
        "Y_test_inv": Y_test_inv,
        "rmse": rmse,
        "mae": mae,
        "mape": mape,
        "r2": r2,
        "mse": mse
    }


### **상품 3, 12 , 42**
* 상품 3: Beverage
* 상품 12: Milk
* 상품 42: Agricultural Products

* **모델링**

In [44]:
# 제품별로 모델 훈련 및 예측 실행
product_ids = [3, 12, 42]
predictions = {}
Y_tests_inv = {}
performance_metrics = {}  # 성능 평가 지표 저장을 위한 딕셔너리

for product_id in product_ids:
    pred_result = train_and_predict(df, product_id)
    predictions[product_id] = pred_result["prediction"]
    Y_tests_inv[product_id] = pred_result["Y_test_inv"]
    # 성능 평가 지표 저장
    performance_metrics[product_id] = {
        "rmse": pred_result["rmse"],
        "mae": pred_result["mae"],
        "mape": pred_result["mape"],
        "r2": pred_result["r2"],
        "mse": pred_result["mse"]
    }


prediction_3 = predictions[3]
Y_test_inv_3 = Y_tests_inv[3]

prediction_12 = predictions[12]
Y_test_inv_12 = Y_tests_inv[12]

prediction_42 = predictions[42]
Y_test_inv_42 = Y_tests_inv[42]

Epoch 1/50
164/164 [==============================] - 14s 29ms/step - loss: 0.0151 - val_loss: 0.0139
Epoch 2/50
164/164 [==============================] - 2s 10ms/step - loss: 0.0100 - val_loss: 0.0133
Epoch 3/50
164/164 [==============================] - 2s 11ms/step - loss: 0.0097 - val_loss: 0.0130
Epoch 4/50
164/164 [==============================] - 2s 11ms/step - loss: 0.0090 - val_loss: 0.0130
Epoch 5/50
164/164 [==============================] - 2s 10ms/step - loss: 0.0086 - val_loss: 0.0126
Epoch 6/50
164/164 [==============================] - 2s 11ms/step - loss: 0.0087 - val_loss: 0.0128
Epoch 7/50
164/164 [==============================] - 2s 13ms/step - loss: 0.0080 - val_loss: 0.0130
Epoch 8/50
164/164 [==============================] - 2s 15ms/step - loss: 0.0082 - val_loss: 0.0130
Epoch 9/50
164/164 [==============================] - 2s 10ms/step - loss: 0.0078 - val_loss: 0.0132
Epoch 10/50
164/164 [==============================] - 2s 11ms/step - loss: 0.0079 - val_l

In [45]:
# 각 제품 ID별로 예측된 판매량과 성능 평가 지표 출력
# 예측 판매량: {predictions[product_id].flatten()}
for product_id in predictions:
    print(f'Product ID {product_id}')
    print(f'RMSE: {performance_metrics[product_id]["rmse"]}, MAE: {performance_metrics[product_id]["mae"]}, MAPE: {performance_metrics[product_id]["mape"]}, R2: {performance_metrics[product_id]["r2"]}, MSE: {performance_metrics[product_id]["mse"]}')
    print('---')

Product ID 3
RMSE: 2289.035916072357, MAE: 1621.826313297923, MAPE: 0.24540025603772264, R2: 0.2526411035317655, MSE: 5239685.425069215
---
Product ID 12
RMSE: 2728.6650276717746, MAE: 1480.4621135432546, MAPE: 0.16866869246432556, R2: 0.24240986056610214, MSE: 7445612.833239008
---
Product ID 42
RMSE: 13.000654033209413, MAE: 10.420250264609733, MAPE: 0.10078783117694351, R2: 0.6858464048541917, MSE: 169.01700529120416
---


## **(2) CNN 모델링**

##### 모델링 코드

In [108]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score

# CNN 모델 훈련 및 예측 함수 수정
def train_and_predict_CNN(df, product_id, test_size=0.2, epochs=100, random_state=None):
    X, Y, scaler_X, scaler_Y = prepare_data(df, product_id)

    # 데이터 분할
    X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=test_size, random_state=random_state)
    X_test = df_test.drop(columns='Qty')
    n_features = X_train.shape[2]

    clear_session()

    model = Sequential([
        Conv1D(32, 5, activation='relu', padding='same', input_shape=(X_train.shape[1], n_features)),
        MaxPooling1D(1),
        Conv1D(64, 5, activation='relu', padding='same'),
        MaxPooling1D(1),
        Conv1D(128, 5, activation='relu', input_shape=(X_train.shape[1], n_features), padding='same'),
        MaxPooling1D(1),
        Conv1D(256, 3, activation='relu', padding='same'),
        Dropout(0.25),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.25),
        Dense(1)
    ])

    # model3.compile(optimizer= Adam(learning_rate = 0.01) ,loss='mse')
    model.compile(optimizer='adam', loss='mean_squared_error')

    # 모델 훈련
    history = model.fit(X_train, Y_train, validation_data = (X_val, Y_val), epochs=epochs, batch_size=4, verbose=1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)]).history

    model_filename = 'model_{}.h5'.format(product_id)  # 제품 ID를 포함한 파일명 생성
    model.save(model_filename)  # 모델 저장
    # 예측 및 예측 값 역정규화
    prediction_scaled = model.predict(X_val)
    prediction = scaler_Y.inverse_transform(prediction_scaled)

    # 실제 값과 예측 값의 RMSE, MAE, MAPE, R², MSE 계산 및 출력
    Y_test_inv = scaler_Y.inverse_transform(Y_val.reshape(-1, 1))
    rmse = np.sqrt(mean_squared_error(Y_test_inv, prediction))
    mae = mean_absolute_error(Y_test_inv, prediction)
    mape = mean_absolute_percentage_error(Y_test_inv, prediction)
    r2 = r2_score(Y_test_inv, prediction)
    mse = mean_squared_error(Y_test_inv, prediction)

    print(f'Product ID {product_id} - RMSE: {rmse}, MAE: {mae}, MAPE: {mape}, R²: {r2}, MSE: {mse}')


    return {
        "prediction": prediction,
        "Y_test_inv": Y_test_inv,
        "rmse": rmse,
        "mae": mae,
        "mape": mape,
        "r2": r2,
        "mse": mse
    }, history


### **상품 3, 12 , 42**
* 상품 3: Beverage
* 상품 12: Milk
* 상품 42: Agricultural Products

In [ ]:
# 제품별로 모델 훈련 및 예측 실행
product_ids = [3, 12, 42]
predictions = {}
Y_tests_inv = {}  # 실제 값 저장을 위한 딕셔너리
model = {}
history = {}

for product_id in product_ids:
    pred_result, history[product_id] = train_and_predict_CNN(df, product_id)
    predictions[product_id] = pred_result["prediction"]
    Y_tests_inv[product_id] = pred_result["Y_test_inv"]

prediction_3 = predictions[3]
Y_test_inv_3 = Y_tests_inv[3]

prediction_12 = predictions[12]
Y_test_inv_12 = Y_tests_inv[12]

prediction_42 = predictions[42]
Y_test_inv_42 = Y_tests_inv[42]

In [ ]:
dl_history_plot(history[3])

In [ ]:
dl_history_plot(history[12])

In [ ]:
dl_history_plot(history[42])

## **(3) DNN 모델링**

##### 모델링 코드

In [107]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten  # Flatten 추가
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score
import numpy as np

# DNN 모델
def train_and_predict(df, product_id, test_size=0.2, random_state=None):
    X, Y, scaler_X, scaler_Y = prepare_data(df, product_id)  # prepare_data 함수는 데이터 준비를 담당합니다.

    X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, test_size=test_size, random_state=random_state)

    model = Sequential([
        Flatten(input_shape=(X_train.shape[1], X_train.shape[2])),
        Dense(256, activation='relu'),
        Dropout(0.2),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(1)
    ])

    model.compile(optimizer='adam', loss='mean_squared_error')

    model.fit(X_train, Y_train, epochs=100, batch_size=1, verbose=1, callbacks=[EarlyStopping(monitor='loss', patience=10)])

    # 모델 저장
    model_filename = 'model_{}.h5'.format(product_id)  # 제품 ID를 포함한 파일명 생성
    model.save(model_filename)  # 모델 저장

    # 예측 및 예측 값 역정규화
    prediction_scaled = model.predict(X_valid)
    prediction = scaler_Y.inverse_transform(prediction_scaled)

    Y_test_inv = scaler_Y.inverse_transform(Y_valid.reshape(-1, 1))

    rmse = np.sqrt(mean_squared_error(Y_test_inv, prediction))
    mae = mean_absolute_error(Y_test_inv, prediction)
    mape = mean_absolute_percentage_error(Y_test_inv, prediction)
    r2 = r2_score(Y_test_inv, prediction)
    mse = mean_squared_error(Y_test_inv, prediction)

    return {
        "prediction": prediction,
        "Y_test_inv": Y_test_inv,
        "rmse": rmse,
        "mae": mae,
        "mape": mape,
        "r2": r2,
        "mse": mse
    }

### **상품 3, 12 , 42**
* 상품 3: Beverage
* 상품 12: Milk
* 상품 42: Agricultural Products

In [ ]:
# 제품 ID 리스트
product_ids = [3, 12, 42]
predictions = {}
Y_tests_inv = {}
performance_metrics = {}  # 성능 평가 지표 저장을 위한 딕셔너리

for product_id in product_ids:
    pred_result = train_and_predict(df, product_id)
    predictions[product_id] = pred_result["prediction"]
    Y_tests_inv[product_id] = pred_result["Y_test_inv"]
    # 성능 평가 지표 저장
    performance_metrics[product_id] = {
        "rmse": pred_result["rmse"],
        "mae": pred_result["mae"],
        "mape": pred_result["mape"],
        "r2": pred_result["r2"],
        "mse": pred_result["mse"]
    }

prediction_3 = predictions[3]
Y_test_inv_3 = Y_tests_inv[3]

prediction_12 = predictions[12]
Y_test_inv_12 = Y_tests_inv[12]

prediction_42 = predictions[42]
Y_test_inv_42 = Y_tests_inv[42]

In [ ]:
# 각 제품 ID별로 예측된 판매량과 성능 평가 지표 출력
for product_id in predictions:
    print(f'Product ID {product_id} 예측 판매량: {predictions[product_id].flatten()}')
    print(f'RMSE: {performance_metrics[product_id]["rmse"]}, MAE: {performance_metrics[product_id]["mae"]}, MAPE: {performance_metrics[product_id]["mape"]}, R2: {performance_metrics[product_id]["r2"]}, MSE: {performance_metrics[product_id]["mse"]}')
    print('-----------------------------------------------------------------------------------------------------------')

# **3.데이터 파이프라인 생성**

* 세부 요구사항
    * 데이터 파이프라인 함수 작성하기
        * input : raw data
        * output : x_test, y_test
        * test set으로 파이프라인 함수를 테스트 해 봅시다.

## **(1) 파이프라인 함수 생성**

In [93]:
def pipeline_split(path, data_list, store_id, product_id):
    path = path
    oil_price = data_list[0]
    orders = data_list[1]
    sales = data_list[2]
    products = data_list[3]
    stores = data_list[4]


    sales['Date'] = pd.to_datetime(sales['Date'] )
    oil_price['Date'] = pd.to_datetime(oil_price['Date'] )
    orders['Date'] = pd.to_datetime(orders['Date'] )

    import holidays
    us_holidays = holidays.UnitedStates()

    nasdaq = pd.read_csv(path+'train/Nasdaq.csv')
    nasdaq.Date = pd.to_datetime(nasdaq.Date)


    unemployment = pd.read_csv(path+'train/Unemployment_usa.csv')
    unemployment['Date'] = pd.to_datetime(unemployment['Date'] )
    unemployment['year'] = unemployment['Date'].dt.year
    unemployment[''] = unemployment['Date'].dt.year

    unemployment.drop(columns='Date', inplace=True)

    master_df = pd.merge(sales, oil_price, on='Date', how='inner')
    master_df = pd.merge(master_df, orders, on=['Store_ID', 'Date'], how='inner')
    master_df = pd.merge(master_df, products, on='Product_ID', how='inner')
    master_df = pd.merge(master_df, stores, on='Store_ID', how='inner')
    master_df = pd.merge(master_df, nasdaq, on='Date', how= 'left')
    master_df['거래량'] = master_df['거래량'].str[:-1].astype('float')
    master_df['month'] = master_df['Date'].dt.month
    master_df['year'] = master_df['Date'].dt.year
    master_df['day_of_week'] = master_df['Date'].dt.dayofweek
    master_df['holiday'] = master_df['Date'].apply(lambda x : x in us_holidays)
    master_df = pd.merge(master_df, unemployment, on=['State', 'year'], how = 'left')
    master_df.drop(columns=[master_df.columns[-1]], inplace=True)  # 컬럼 삭제
    master_df['before_2days'] = master_df['Qty'].shift(periods=2)
    def remove_comma_and_convert_to_float(s):
        if isinstance(s, str):
            s_without_comma = s.replace(',', '')
            return float(s_without_comma)
        else:
            return s

    str_list = ['종가', '시가', '고가', '저가']
    for s in str_list:
        master_df[s] = master_df[s].apply(remove_comma_and_convert_to_float)
    master_df['변동 %'] = master_df['변동 %'].str[:-1].astype('float')
    isnull_list = ['WTI_Price', '종가', '시가', '고가', '저가', '거래량', '변동 %', 'before_2days']
    for isnull in isnull_list:
        master_df[isnull] = master_df[isnull].interpolate(method='linear')
    for isnull in isnull_list:
        master_df[isnull] = master_df[isnull].fillna(master_df[isnull].mean())

    master_df = master_df.loc[(master_df['Product_ID'] == product_id) & (master_df['Store_ID']==store_id)]

    y_test = master_df['Qty']
    x_test = master_df.drop(columns=['Date', 'Qty'])


    return x_test, y_test

## **(2) test 데이터로 확인**

* test용 데이터셋들 로딩

In [94]:
oil_price = pd.read_csv(path + 'test/oil_price_test.csv')
orders = pd.read_csv(path + 'test/orders_test.csv')
sales = pd.read_csv(path + 'test/sales_test.csv')
products = pd.read_csv(path + 'test/products.csv')
stores = pd.read_csv(path + 'test/stores.csv')
data_list = [oil_price, orders, sales, products, stores]
x_test, y_test = pipeline_split(path, data_list, 44, 3)

* 예측해보기

##### **LSTM 모델링**

In [101]:
from tensorflow.keras.models import load_model
def load_and_predict(product_id, df_test):
    # 데이터 준비
    X_test, Y_test, _, scaler_Y = prepare_data(df_test, product_id)  # prepare_data 함수는 데이터를 준비하는 사용자 정의 함수입니다.

    # 모델 불러오기
    model_filename = 'model_{}.h5'.format(product_id)
    model = load_model(model_filename)

    # 예측
    prediction_scaled = model.predict(X_test)
    prediction = scaler_Y.inverse_transform(prediction_scaled)  # 예측값 역정규화
    Y_test_inv = scaler_Y.inverse_transform(Y_test.reshape(-1, 1))  # 실제 값 역정규화

    # 성능 지표 계산
    rmse = np.sqrt(mean_squared_error(Y_test_inv, prediction))
    mae = mean_absolute_error(Y_test_inv, prediction)
    mape = mean_absolute_percentage_error(Y_test_inv, prediction)
    r2 = r2_score(Y_test_inv, prediction)

    print(f"Product ID: {product_id}")
    print(f"RMSE: {rmse}, MAE: {mae}, MAPE: {mape}, R^2: {r2}\n")

In [102]:
product_ids = [3, 12, 42]

for pid in product_ids:
    load_and_predict(pid, df_test)


1/1 [==============================] - 2s 2s/step
Product ID: 3
RMSE: 1561.6918875173603, MAE: 1201.025390625, MAPE: 0.12547721139541188, R^2: -0.6283609911450891

1/1 [==============================] - 2s 2s/step
Product ID: 12
RMSE: 2160.7815045640787, MAE: 1783.0844029017858, MAPE: 0.18639421728770916, R^2: -1.3292135211105895

1/1 [==============================] - 2s 2s/step
Product ID: 42
RMSE: 9.78397551654677, MAE: 7.297310602097283, MAPE: 0.07217974418753265, R^2: 0.19748984836517158



#### **CNN 모델링**

In [ ]:
from tensorflow.keras.models import load_model
def load_and_predict(product_id, df_test):
    # 데이터 준비
    X_test, Y_test, _, scaler_Y = prepare_data(df_test, product_id)  # prepare_data 함수는 데이터를 준비하는 사용자 정의 함수입니다.

    # 모델 불러오기
    model_filename = 'model_CNN{}.h5'.format(product_id)
    model = load_model(model_filename)

    # 예측
    prediction_scaled = model.predict(X_test)
    prediction = scaler_Y.inverse_transform(prediction_scaled)  # 예측값 역정규화
    Y_test_inv = scaler_Y.inverse_transform(Y_test.reshape(-1, 1))  # 실제 값 역정규화

    # 성능 지표 계산
    rmse = np.sqrt(mean_squared_error(Y_test_inv, prediction))
    mae = mean_absolute_error(Y_test_inv, prediction)
    mape = mean_absolute_percentage_error(Y_test_inv, prediction)
    r2 = r2_score(Y_test_inv, prediction)

    print(f"Product ID: {product_id}")
    print(f"RMSE: {rmse}, MAE: {mae}, MAPE: {mape}, R^2: {r2}\n")

In [ ]:
load_and_predict(12, df_test)

#### **DNN 모델링**

In [ ]:
from tensorflow.keras.models import load_model
def load_and_predict(product_id, df_test):
    # 데이터 준비
    X_test, Y_test, _, scaler_Y = prepare_data(df_test, product_id)  # prepare_data 함수는 데이터를 준비하는 사용자 정의 함수입니다.

    # 모델 불러오기
    model_filename = 'model_{}.h5'.format(product_id)
    model = load_model(model_filename)

    # 예측
    prediction_scaled = model.predict(X_test)
    prediction = scaler_Y.inverse_transform(prediction_scaled)  # 예측값 역정규화
    Y_test_inv = scaler_Y.inverse_transform(Y_test.reshape(-1, 1))  # 실제 값 역정규화

    # 성능 지표 계산
    rmse = np.sqrt(mean_squared_error(Y_test_inv, prediction))
    mae = mean_absolute_error(Y_test_inv, prediction)
    mape = mean_absolute_percentage_error(Y_test_inv, prediction)
    r2 = r2_score(Y_test_inv, prediction)

    print(f"Product ID: {product_id}")
    print(f"RMSE: {rmse}, MAE: {mae}, MAPE: {mape}, R^2: {r2}\n")

# **4.비즈니스 평가**

* 세부 요구사항
    * 기본 시뮬레이션 함수가 제공됩니다.
        * 필요하다면 해당 함수를 수정해서 사용할 수 있습니다.
    * 시뮬레이션 함수 사용하여 재고 평가
        * 실제값, 예측값, 안전재고 수량을 넣고 일평균 재고 금액을 확인합니다.
        * 기회손실수량은 0으로 맞추도록 안전재고 수량을 조절합니다.
        * 평균재고금액을 확인합니다.
    * 데이터는 test 셋을 이용합니다.
        * 예측 후 재고 평가 시, 스케일링된 y를 원래대로 돌려 놓아야 합니다.
            * y_real = y_sacled * (y_max - y_min) + y_min
            * pred_real = pred * (y_max - y_min) + y_min

## **(1) 재고 시뮬레이터**

In [103]:
def inv_simulator(y, pred, safe_stock, price) :

    # 시뮬레이션 df 틀 만들기
    temp = pd.DataFrame({'y':y.reshape(-1,), 'pred':pred.reshape(-1,).round()})

    temp['base_stock'] = 0
    temp['close_stock'] = 0
    temp['order'] = 0
    temp['receive'] = 0

    # 시뮬레이션

    for i in range(len(temp)-2):  # 발주량은 leadtime 후 판매 예측량에 기초하므로 계산을 위해 마지막 leadtime 만큼의 행 제외
        if i == 0 : #첫 행. 2일 전 데이터가 없으므로,
            temp.loc[ i ,'receive'] = temp.loc[ i ,'y']  # 입고량은 실판매량으로 계산
            temp.loc[ i ,'base_stock'] = temp.loc[ i ,'receive'] + safe_stock  # 기초재고는 실판매량 + 안전재고로 계산

        elif i == 1 : # 둘째 행, 2일 전 데이터가 없음.
            temp.loc[ i ,'receive'] = temp.loc[ i ,'y'] # 입고량은 실판매량으로 계산
            temp.loc[ i ,'base_stock'] = temp.loc[ i ,'receive'] + temp.loc[ i-1 ,'close_stock']
        else :      # 나머지 전체 행.
            temp.loc[ i ,'receive'] = temp.loc[ i-2 ,'order']    # 입고량 = 2일전 발주량
            temp.loc[ i ,'base_stock'] = temp.loc[ i ,'receive'] + temp.loc[ i-1 ,'close_stock']  # 기초재고 = 입고량 + 전날 기말재고

        # 기말재고 = 기초재고 - 판매량,  만약 0보다 작으면 0으로.
        stock = round(temp.loc[i, 'base_stock'] - temp.loc[i, 'y'])
        temp.loc[i, 'close_stock'] = np.where(stock> 0, stock, 0)

        # 발주량 = 2일후 판매예측량 + 안전재고 - 기말재고,  만약 주문량이 0보다 작으면 0
        order = temp.loc[i+2, 'pred'] + safe_stock - temp.loc[i, 'close_stock']
        temp.loc[i, 'order'] = np.where(order>0,order, 0)

    # 기회손실 = 만약 (기초재고 - 실판매량)이 0보다 작으면, 그만큼이 기회손실
    temp['lost'] = np.where((temp['base_stock'] - temp['y'])<0, (temp['base_stock'] - temp['y']),0).round()

    inventory = temp[:len(temp)-2]

    # 측정지표 계산
    DailyStock = ((inventory['base_stock'] + inventory['close_stock'])/2)
    DailyTurnover = (inventory['y'] + inventory['lost']) / DailyStock

    AvgDailyStock = round(DailyStock.mean(),3)
    AvgDailyStockAmt = AvgDailyStock * price
    turnover = round(DailyTurnover.mean(), 3)
    lost_sum = inventory['lost'].sum()


    print(f'일평균 재고량     : {AvgDailyStock}')
    print(f'일평균 재고 금액  : {AvgDailyStockAmt}')
    print(f'일평균 재고회전율 : {turnover}')
    print(f'기회손실 수량     : {lost_sum}')

    return inventory

## **(2) 예측 결과 시뮬레이션(재고금액 평가)**


In [81]:
a_3 = inv_simulator(Y_test_inv_3, predictions[3], 4300, 8)
a_3

일평균 재고량     : 8847.556
일평균 재고 금액  : 70780.448
일평균 재고회전율 : 0.951
기회손실 수량     : -66120.0


,y,pred,base_stock,close_stock,order,receive,lost
0,7887.0,6454.0,12187,4300,8471,7887,0.0
1,7884.0,9083.0,12184,4300,5329,7884,0.0
2,6115.0,8471.0,12771,6656,2685,8471,0.0
3,5484.0,5329.0,11985,6501,5014,5329,0.0
4,3989.0,5041.0,9186,5197,6841,2685,0.0
...,...,...,...,...,...,...,...
157,5762.0,6256.0,11143,5381,4107,1644,0.0
158,5212.0,6725.0,6907,1695,11376,1526,0.0
159,4280.0,5188.0,5802,1522,11984,4107,0.0
160,8482.0,8771.0,12898,4416,7954,11376,0.0


In [104]:
a_12 = inv_simulator(Y_test_inv_12, predictions[12], 10000, 6)
a_12

일평균 재고량     : 14888.497
일평균 재고 금액  : 89330.98199999999
일평균 재고회전율 : 0.683
기회손실 수량     : -13276.0


,y,pred,base_stock,close_stock,order,receive,lost
0,10351.0,10916.0,20351,10000,8197,10351,0.0
1,8312.0,10672.0,18312,10000,8900,8312,0.0
2,6781.0,8197.0,18197,11416,7801,8197,0.0
3,7374.0,8900.0,20316,12942,5579,8900,0.0
4,6872.0,9217.0,20743,13871,11717,7801,0.0
...,...,...,...,...,...,...,...
157,15577.0,11839.0,24064,8487,10856,20064,0.0
158,7625.0,9730.0,24217,16592,2833,15730,0.0
159,9834.0,9343.0,27448,17614,1335,10856,0.0
160,7938.0,9425.0,20447,12509,5564,2833,0.0


In [106]:
a_42 = inv_simulator(Y_test_inv_42, predictions[42], 210, 5)
a_42

일평균 재고량     : 259.62
일평균 재고 금액  : 1298.1
일평균 재고회전율 : 0.423
기회손실 수량     : 0.0


,y,pred,base_stock,close_stock,order,receive,lost
0,141.0,132.0,351,210,116,141.0,0.0
1,69.0,79.0,279,210,76,69.0,0.0
2,112.0,116.0,326,214,62,116.0,0.0
3,92.0,76.0,290,198,107,76.0,0.0
4,82.0,66.0,260,178,121,62.0,0.0
...,...,...,...,...,...,...,...
157,99.0,86.0,364,265,68,214.0,0.0
158,105.0,93.0,418,313,20,153.0,0.0
159,132.0,123.0,381,249,101,68.0,0.0
160,130.0,123.0,269,139,158,20.0,0.0
